In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import csv

import datetime

from datetime import date, timedelta
import dateutil.relativedelta
from dateutil import relativedelta

import os
# os.listdir('./Data/')

### Economic Indicators Data

- Gross Domestic Product (GDP)
- Federal Funds Effective Rate (FEDFUNDS)
- Inflation (INFL)
- Personal Consumption Expenditure (PCE)
- Consumer Price Index (CPI)
- Federal Debt (GFDEBTN)
- Federal Tax Revenue (FEDREV)
- Personal Saving Rate (PSAVERT)
- Revolving Consumer Credit Outstanding (REVOLSL)
- Total Consumer Credit Outstanding (TOTALSL)
- Unemployment Rate (UNRATE)
- Swizterland Gross Domestic Product (SGDP)

In [2]:
# Import Datasets
dataset_dict = {'CPIAUCSL':'cpi_df','GDP':'gdp_df', 'FEDREV':'fedrev_df', 'PCE':'pce_df', 
    'FEDFUNDS':'fedfunds_df', 'GFDEBTN':'feddebt_df', 'PSAVERT':'psave_df', 'REVOLSL':'revolcredit_df', 
    'TOTALSL':'cc_outstanding_df','UNRATE':'unemploy_df','SGDP':'sgdp_df',}

In [3]:
# Create Individual Datasets
for k, v in dataset_dict.items():
    globals()[v] = pd.read_csv('{}.csv'.format(k))

<strong>Individual Datsets</strong>
- GDP: gdp_df
- FEDFUNDS: fedfunds_df
- INFL (derivative of CPI): infl_df
- PCE: pce_df
- CPI: cpi_df
- DEBT: feddebt_df
- FEDREV: fedrev_df
- PSAVE: psave_df
- REVOLSL: revolcredit_df
- TOTALSL: cc_outstanding_df
- UNRATE: unemploy_df
- SGDP: sgdp_df

In [4]:
# Transform Datasets
# Rename Columns: CPI, FEDREV
cpi_df = cpi_df.rename(columns={'CPIAUCSL': 'CPI'})
fedrev_df = fedrev_df.rename(columns={'Total Direct Revenue - Federal $ million nominal': 'FEDREV_mm_nom'})

# Transform Dataset: FEDREV
# Create YYYY-MM-DD Date Column
fedrev_df['DATE'] = fedrev_df['Year'] 
fedrev_df['DATE'] = fedrev_df['DATE'].astype(str)
fedrev_df['DATE'] = fedrev_df['DATE'] + '-01-01'
fedrev_df = fedrev_df[['DATE', 'FEDREV_mm_nom']]

# Create Dataset: INFL: Inflation
# Calculate 12 month lag for later inflation % calculations
cpi_df['lag_12_diff'] = cpi_df['CPI'].diff(periods=12)
cpi_df['lag_12'] = cpi_df['CPI'] - cpi_df['lag_12_diff']
cpi_df['INFL'] = (cpi_df['lag_12_diff'] / cpi_df['lag_12']) * 100

# INF: Inflation
infl_df = cpi_df[['DATE', 'INFL']].dropna()
cpi_df = cpi_df[['DATE', 'CPI']]

# Add INFL to dataset_dict
dataset_dict['INFL'] = 'infl_df'

# Transform SGDP Date
sgdp_df['DATE'] = pd.to_datetime(sgdp_df['DATE'])
sgdp_df['DATE'] = sgdp_df['DATE'].dt.strftime('%Y-%m-%d')

<strong>Merged Dataset</strong>

In [5]:
# Create Merged Dataset
dates_lst = []
[dates_lst.extend(globals()[v].iloc[:,0]) for k, v in dataset_dict.items()]
dates_lst = list(set(dates_lst))
dates_lst.sort()

merged_df = pd.DataFrame(dates_lst, columns=['DATE'])
for k, v in dataset_dict.items():
    merged_df = merged_df.merge(globals()[v], on='DATE', how='outer')

merged_df.tail(20)

,DATE,CPI,GDP,FEDREV_mm_nom,PCE,FEDFUNDS,GFDEBTN,PSAVERT,REVOLSL,TOTALSL,UNRATE,SGDP,INFL
1018,2022-06-01,294.728,NaN,NaN,17550.0,1.21,NaN,2.7,1134.71289,4724.93980,3.6,NaN,8.932987
1019,2022-07-01,294.628,25994.639,NaN,17556.4,1.68,30928912.0,3.5,1147.00357,4751.63715,3.5,NaN,8.413182
1020,2022-08-01,295.320,NaN,NaN,17692.2,2.33,NaN,3.2,1161.98082,4779.19999,3.7,NaN,8.227361
1021,2022-09-01,296.539,NaN,NaN,17804.0,2.56,NaN,3.0,1171.50821,4806.78059,3.5,NaN,8.214854
1022,2022-10-01,297.987,26408.405,NaN,17915.1,3.08,31419689.0,3.0,1185.50873,4841.94131,3.7,NaN,7.762493
1023,2022-11-01,298.598,NaN,NaN,17892.5,3.78,NaN,3.3,1201.09011,4874.67573,3.6,NaN,7.135348
1024,2022-12-01,298.990,NaN,NaN,17943.6,4.10,NaN,3.4,1212.60894,4894.04253,3.5,NaN,6.444940
1025,2022-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,807.706035,NaN
1026,2023-01-01,300.536,26813.601,4439280.0,18229.6,4.33,31458438.0,4.4,1223.01589,4915.00879,3.4,NaN,6.347156
1027,2023-02-01,301.648,NaN,NaN,18296.5,4.57,NaN,4.7,1226.38197,4926.27309,3.6,NaN,5.986438


### War Dates Dataframe

In [6]:
# Save War Dates Dataframe
war_dates_df = pd.read_csv('war_dates.csv')
war_dates_df['War_Start_Date'] = pd.to_datetime(war_dates_df['War_Start_Date'])
war_dates_df['War_End_Date'] = pd.to_datetime(war_dates_df['War_End_Date'])

# Create War Timeline Dataframe
timeline_start = min(war_dates_df.War_Start_Date)
print(timeline_start)
timeline_dates = pd.date_range(timeline_start-timedelta(weeks=5),datetime.datetime.now()+timedelta(weeks=5),freq='d')
war_timeline_df = pd.DataFrame(timeline_dates, columns=['DATE'])
war_timeline_df['War'] = np.nan 
war_timeline_df['War_Major'] = np.nan 

1939-09-01 00:00:00


In [7]:
# Update War Timeline Dataframe
for i, row in war_timeline_df.iterrows():
    for i2, row2 in war_dates_df.iterrows():
        if (war_timeline_df['DATE'].iloc[i] <= war_dates_df.War_End_Date.iloc[i2]) & (war_timeline_df['DATE'].iloc[i] >= war_dates_df.War_Start_Date.iloc[i2]):
            war_timeline_df['War'].iloc[i] = war_dates_df.War.iloc[i2]
            if war_dates_df.War_Major.iloc[i2] == 'Yes':
                war_timeline_df['War_Major'].iloc[i] = war_dates_df.War_Major.iloc[i2]

C:\Users\Julio\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [8]:
# Updtate War Timeline Dataframe for Visualizations
war_timeline_df['War_Major_Text'] = np.nan 
war_timeline_df.loc[war_timeline_df.War.isnull(), 'War_Major_Text'] = 'U.S. Not Engaged in War'
war_timeline_df.loc[war_timeline_df.War_Major=='Yes', 'War_Major_Text'] = 'U.S. Engaged in Major War'
war_timeline_df.War_Major_Text.fillna('U.S. Engaged in War', inplace=True)
war_timeline_df

,DATE,War,War_Major,War_Major_Text
0,1939-07-28,NaN,NaN,U.S. Not Engaged in War
1,1939-07-29,NaN,NaN,U.S. Not Engaged in War
2,1939-07-30,NaN,NaN,U.S. Not Engaged in War
3,1939-07-31,NaN,NaN,U.S. Not Engaged in War
4,1939-08-01,NaN,NaN,U.S. Not Engaged in War
...,...,...,...,...
30851,2024-01-14,NaN,NaN,U.S. Not Engaged in War
30852,2024-01-15,NaN,NaN,U.S. Not Engaged in War
30853,2024-01-16,NaN,NaN,U.S. Not Engaged in War
30854,2024-01-17,NaN,NaN,U.S. Not Engaged in War


<strong>Save Datasets to CSV & Store Variables</strong>

In [9]:
# Save Datasets to CSV
merged_df.to_csv('merged_df.csv', index=False)   
infl_df.to_csv('INFL.csv', index=False)  

# Add 3 Global Variables
global cc_outstanding_df, war_timeline_df, war_dates_df

# Store Variables
# dataset_dict.values()
%store cpi_df gdp_df fedrev_df pce_df fedfunds_df feddebt_df psave_df revolcredit_df unemploy_df infl_df cc_outstanding_df merged_df dataset_dict war_timeline_df war_dates_df sgdp_df

Stored 'cpi_df' (DataFrame)
Stored 'gdp_df' (DataFrame)
Stored 'fedrev_df' (DataFrame)
Stored 'pce_df' (DataFrame)
Stored 'fedfunds_df' (DataFrame)
Stored 'feddebt_df' (DataFrame)
Stored 'psave_df' (DataFrame)
Stored 'revolcredit_df' (DataFrame)
Stored 'unemploy_df' (DataFrame)
Stored 'infl_df' (DataFrame)
Stored 'cc_outstanding_df' (DataFrame)
Stored 'merged_df' (DataFrame)
Stored 'dataset_dict' (dict)
Stored 'war_timeline_df' (DataFrame)
Stored 'war_dates_df' (DataFrame)
Stored 'sgdp_df' (DataFrame)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ba1e4b21-f3e5-4d67-887d-75e6f1b74e8f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>